In [24]:
# load labraries 
import pandas as pd 
import numpy as np 

In [26]:
import pandas as pd

# Load with latin1 encoding to handle special characters (£, ₦, etc.)
data = pd.read_csv("BusinessFundingData.csv", encoding="latin1")

# Preview
print("Shape:", data.shape)
data.head()


Shape: (26, 11)


,Website Domain,Effective date,Found At,Financing Type,Financing Type Normalized,Categories,Investors,Investors Count,Amount,Amount Normalized,Source Urls
0,trafigura.com,NaN,2024-03-14T01:00:00+01:00,NaN,NaN,[],NaN,NaN,$1.9b,1900000000,https://www.tradefinanceglobal.com/posts/trafi...
1,zenobe.com,NaN,2024-05-31T02:00:00+02:00,NaN,NaN,[],"avivainvestors.com, lloydsbankinggroup.com, sa...",9.0,$522.7 million,522700000,https://realassets.ipe.com/news/aviva-among-le...
2,zenobe.com,NaN,2024-07-24T02:00:00+02:00,NaN,NaN,"[""private_equity""]",NaN,NaN,£41.7m,53671000,https://www.innovationnewsnetwork.com/zenobe-a...
3,canva.com,NaN,2024-05-01T02:00:00+02:00,NaN,NaN,[],stackcapitalgroup.com,1.0,US$8 million,8000000,https://www.globenewswire.com/news-release/202...
4,fidelity.com,NaN,2024-04-11T02:00:00+02:00,NaN,NaN,[],chevychasetrust.com,1.0,$1.96 million,1960000,https://www.defenseworld.net/2024/04/11/chevy-...


``` Data Loading Observations

  The dataset could not be loaded with the default UTF 8 encoding because it contains special characters 
  Using encoding="latin1" solved the problem and allowed the file to load successfully.  
  The dataset has 26 rows and 11 columns.  
  At this stage, the raw data is ready for further quality checks and preprocessing.```


In [ ]:
# Check missing values in each column
data.isna().sum()

Website Domain                0
Effective date               20
Found At                      0
Financing Type               18
Financing Type Normalized    18
Categories                    0
Investors                    13
Investors Count              13
Amount                        0
Amount Normalized             0
Source Urls                   0
dtype: int64

In [28]:
# Check column data types and non-null counts
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Website Domain             26 non-null     object 
 1   Effective date             6 non-null      object 
 2   Found At                   26 non-null     object 
 3   Financing Type             8 non-null      object 
 4   Financing Type Normalized  8 non-null      object 
 5   Categories                 26 non-null     object 
 6   Investors                  13 non-null     object 
 7   Investors Count            13 non-null     float64
 8   Amount                     26 non-null     object 
 9   Amount Normalized          26 non-null     int64  
 10  Source Urls                26 non-null     object 
dtypes: float64(1), int64(1), object(9)
memory usage: 2.4+ KB


In [29]:
# Check number of unique values per column
data.nunique()

Website Domain               21
Effective date                6
Found At                     23
Financing Type                5
Financing Type Normalized     5
Categories                    9
Investors                    13
Investors Count               4
Amount                       26
Amount Normalized            24
Source Urls                  26
dtype: int64

``` ###  Data Quality Checks
 Missing Values:
   `Effective date` has mostly missing values.
   `Financing Type` and `Financing Type Normalized` also contain many missing entries.
   Other columns are more complete.

Data Types:
  `Amount` is stored as an object (string) even though it represents money.
   `Found At` and `Effective date` are stored as strings instead of datetime.
   `Investors` is a text field with multiple names, while `Investors Count` is numeric.

 Unique Values:
   Columns like `Website Domain` and `Categories` have several repeated entries.
   `Amount Normalized` is already numeric and can serve as a reference when cleaning `Amount`.

 Conclusion:  
The dataset is small (26 rows) but needs cleaning:
 Dates must be converted into datetime format.
 `Amount` must be cleaned into numeric.
 `Investors` and `Categories` should be structured properly.```


In [30]:
# Standardize column names
data.columns = (
    data.columns.str.strip()  # remove leading/trailing spaces
             .str.lower()   # make all lowercase
             .str.replace(r'[^0-9a-zA-Z]+', '_', regex=True)  # replace spaces/symbols with _
             .str.strip('_')  # remove leading/trailing underscores
)
data.columns


Index(['website_domain', 'effective_date', 'found_at', 'financing_type',
       'financing_type_normalized', 'categories', 'investors',
       'investors_count', 'amount', 'amount_normalized', 'source_urls'],
      dtype='object')

``` Column Name Standardization

 All column names have been converted to **lowercase** for consistency.
 Spaces and special characters were replaced with **underscores** (e.g., `Website Domain` → `website_domain`).
 Leading and trailing underscores were removed.
 This makes it easier to reference columns in code without errors.

Example:  
`Website Domain` → `website_domain`  
`Effective date` → `effective_date`  
`Amount Normalized` → `amount_normalized` ```


In [31]:
# Convert date columns to datetime 
for col in ['found_at', 'effective_date']:
    if col in data.columns:
        data[col] = pd.to_datetime(data[col], errors='coerce')



C:\Users\USER\AppData\Local\Temp\ipykernel_1860\279617900.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data[col] = pd.to_datetime(data[col], errors='coerce')


``` Date Handling

 Both `found_at` and `effective_date` columns were converted from **string** to **datetime** format.  
 `errors="coerce"` was used to safely handle invalid or missing values, turning them into `NaT` (Not a Time).  
 This conversion makes it easier to:
   Perform **time based analysis** (e.g., trends over time).  
   Calculate time differences (e.g., funding gap between events).  

Example:  
`2024-03-14T01:00:00+01:00` → `2024-03-14 01:00:00` (datetime object)``` 


In [35]:
import ast, re

# Function to clean and parse categories
def parse_categories(x):
    if pd.isna(x):
        return []
    s = str(x).strip()
    if s == "[]":
        return []
    # Try to parse stringified list
    try:
        val = ast.literal_eval(s)
        if isinstance(val, list):
            return [str(v).strip().lower() for v in val]
    except (ValueError, SyntaxError):
        pass
    # Fallback: split by comma or semicolon
    return [p.strip().lower() for p in re.split(r"[;,]", s) if p.strip()]

# Apply function
data["categories_list"] = data["categories"].apply(parse_categories)
data["n_categories"] = data["categories_list"].apply(len)

# Preview result
data[["categories", "categories_list", "n_categories"]].head()


,categories,categories_list,n_categories
0,[],[],0
1,[],[],0
2,"[""private_equity""]",[private_equity],1
3,[],[],0
4,[],[],0


``` Cleaning Categories

 The `categories` column originally stored values as text that looked like lists (e.g., `["private_equity"]`) or as empty brackets (`[]`).  
 Using a custom function, we:
   Parsed stringified lists into **true Python lists**.  
   Normalized text to **lowercase** and removed extra spaces.  
   Counted the number of categories per row in a new column `n_categories`.  

✅ Example:  
`["private_equity"]` → `['private_equity']` with `n_categories = 1`  
`[]` → `[]` with `n_categories = 0`
```

In [36]:
# split and clean investors
def split_investors(text):
    if pd.isna(text) or str(text).strip() == "":
        return []
    # Split by commas, 'and', or '&'
    parts = re.split(r",| and | & ", str(text), flags=re.IGNORECASE)
    cleaned = [p.strip() for p in parts if p.strip()]
    # Deduplicate while keeping order
    seen = set()
    unique = []
    for p in cleaned:
        if p.lower() not in seen:
            seen.add(p.lower())
            unique.append(p)
    return unique

# Apply function
data["investors_list"] = data["investors"].apply(split_investors)
data["investors_count_derived"] = data["investors_list"].apply(len)

# Compare with original investors_count column
data[["investors", "investors_list", "investors_count", "investors_count_derived"]].head()


,investors,investors_list,investors_count,investors_count_derived
0,NaN,[],NaN,0
1,"avivainvestors.com, lloydsbankinggroup.com, sa...","[avivainvestors.com, lloydsbankinggroup.com, s...",9.0,9
2,NaN,[],NaN,0
3,stackcapitalgroup.com,[stackcapitalgroup.com],1.0,1
4,chevychasetrust.com,[chevychasetrust.com],1.0,1


``` Cleaning Investors

 The `investors` column was originally a long text string with multiple investor names separated by commas, "and", or "&".  
 We cleaned it by:
   Splitting text into **individual investor names**.  
   Stripping extra spaces and normalizing case.  
   Removing duplicates while keeping the first occurrence.  
 A new column `investors_list` stores investors as a Python list.  
 A derived column `investors_count_derived` was added to compare with the original `investors_count`.  

✅ Example:  
`"avivainvestors.com, lloydsbankinggroup.com, santander.com"`  
→ `['avivainvestors.com', 'lloydsbankinggroup.com', 'santander.com']` with `investors_count_derived = 3`
```

In [37]:
# clean and parse 'amount' values
def parse_amount(value):
    if pd.isna(value):
        return np.nan
    s = str(value).lower().replace(",", "").strip()

    # Multiplier for scale
    mult = 1
    if "billion" in s:
        mult = 1e9
        s = s.replace("billion", "")
    elif "million" in s:
        mult = 1e6
        s = s.replace("million", "")
    elif s.endswith("b"):
        mult = 1e9
        s = s[:-1]
    elif s.endswith("m"):
        mult = 1e6
        s = s[:-1]

    # Remove currency symbols
    s = re.sub(r"[^0-9.]", "", s)

    # Convert to float
    return float(s) * mult if s else np.nan

# Apply function
data["amount_parsed"] = data["amount"].apply(parse_amount)

# Compare original vs parsed vs normalized
data[["amount", "amount_parsed", "amount_normalized"]].head()


,amount,amount_parsed,amount_normalized
0,$1.9b,1.900000e+09,1900000000
1,$522.7 million,5.227000e+08,522700000
2,£41.7m,4.170000e+07,53671000
3,US$8 million,8.000000e+06,8000000
4,$1.96 million,1.960000e+06,1960000


``` Cleaning Amounts

 The `amount` column was inconsistent, with formats such as `$1.9b`, `£41.7m`, `US$8 million`.  
 We built a parser that:
   Recognizes suffixes (`b`, `m`, `billion`, `million`).  
   Removes currency symbols ($, £, ₦, €).  
   Converts everything into a clean **numeric value** (`amount_parsed`).  
 We then compared it with the provided `amount_normalized` to ensure consistency.  

✅ Example:  
 `$1.9b` → `1,900,000,000`  
 `£41.7m` → `41,700,000`  
 `US$8 million` → `8,000,000`
```

In [38]:
#Save the cleaned dataset
data.to_csv("business_ataunding_cleaned.csv", index=False)

print("✅ Cleaned dataset saved successfully as 'business_funding_cleaned.csv'")


✅ Cleaned dataset saved successfully as 'business_funding_cleaned.csv'


``` Saving the Cleaned Dataset

 After cleaning and transforming the data, we exported it to a new file called **`business_funding_cleaned.csv`**.  
 This file is now:
   Free from messy formats in `Amount`.  
   Equipped with structured lists for `Categories` and `Investors`.  
   Has proper datetime columns for `found_at` and `effective_date`.  
   Includes derived helper columns such as `n_categories`, `investors_list`, `investors_count_derived`, and `amount_parsed`.  
 This dataset is **ready for analysis** in further assignments or machine learning projects.
```